# Answering Business Questions Using SQL

### Certain questions were given to us by Dataquest. We will be focusing on answering these questions.

#### We will be using a modified version of the database [Chinook](https://github.com/lerocha/chinook-database). Chinook is a database that contains information about a fictional digital music shop. The information is contained in 11 tables. Here's a diagram of the database (Source: Dataquest Mission 189):
![](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

Let's connect to the database and familiarize ourselves with it:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Moving on to the first question:

"The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA."

#### Let's write a query that will help us find out which genres sell the most in the USA:

In [3]:
%%sql
WITH genre_tracks AS (
                      SELECT t.track_id,
                             g.name AS genre
                        FROM track AS t
                       INNER JOIN genre AS g ON g.genre_id = t.genre_id
                     ),
       usa_tracks AS (
                      SELECT il.track_id,
                             SUM(il.quantity) AS tracks_sold
                        FROM customer c
                       INNER JOIN invoice i ON i.customer_id = c.customer_id
                       INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
                       WHERE c.country = "USA"
                       GROUP BY 1
                     )
SELECT g.genre,
       SUM(u.tracks_sold) AS tracks_sold_in_usa,
       SUM(u.tracks_sold) / (SELECT TOTAL(tracks_sold)
                               FROM usa_tracks) AS percentage
  FROM genre_tracks AS g
 INNER JOIN usa_tracks u ON u.track_id = g.track_id
 GROUP BY 1
 ORDER BY 2 DESC;

Done.


genre,tracks_sold_in_usa,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


As we can see above, Rock tracks have the most sales followed by Alternative  Punk. If we compare all these values, the three albums that will be added to the store should belong to Red Tone, Slim Jim Bites and Meteor and the Girls respectively.

### Second question:

"Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others."

#### We should write a query that groups customer purchases by employees. We will be adding the dates the employees were hired and the latest invoice date to make it a fair comparison.  

In [4]:
%%sql
WITH employee_sales AS (SELECT e.first_name || " " || e.last_name AS employee,
                               SUM(i.total) AS total_sales,
                               e.employee_id,
                               (SELECT MAX(invoice_date) FROM invoice GROUP BY customer_id) AS date
                          FROM customer c
                         INNER JOIN employee e ON c.support_rep_id = e.employee_id
                         INNER JOIN invoice i ON i.customer_id = c.customer_id
                         GROUP BY 1)

SELECT e.hire_date,
       es.employee,
       es.total_sales,
       es.date AS current_date
  FROM employee e
 INNER JOIN employee_sales es ON es.employee_id = e.employee_id
 ORDER BY 1;

Done.


hire_date,employee,total_sales,current_date
2017-04-01 00:00:00,Jane Peacock,1731.5100000000039,2020-07-24 00:00:00
2017-05-03 00:00:00,Margaret Park,1584.0000000000034,2020-07-24 00:00:00
2017-10-17 00:00:00,Steve Johnson,1393.920000000002,2020-07-24 00:00:00


We sorted the table by the dates they were hired. As their service time increases, so does their total sales. So, our current date is 24.07.2020 according to the invoice table. If we do some quick maths, their average monthly sales are approximately:

* Jane: 43
* Margaret: 41
* Steve: 42

They all seem to be performing pretty much the same.

### Next, we will analyze the sales data for different countries.

###### To avoid overcrowding and make our table more readable, we will be arranging countries with only 1 customer into an "Other" row and we will be moving it to the very bottom of the table.

In [5]:
%%sql
WITH other_countries AS (SELECT CASE
                                    WHEN (SELECT COUNT(*)
                                            FROM customer
                                           WHERE country = c.country) = 1 THEN "Other"
                                    ELSE c.country
                                 END AS country,
                                 i.*
                           FROM customer c
                          INNER JOIN invoice i ON i.customer_id = c.customer_id)

SELECT country, total_customers, total_sales, avg_sales_per_customer, avg_order_value
  FROM (SELECT country,
               COUNT(DISTINCT customer_id) total_customers,
               TOTAL(total) total_sales,
               TOTAL(total)/COUNT(DISTINCT customer_id) avg_sales_per_customer,
               TOTAL(total)/COUNT(*) avg_order_value,
               CASE
                   WHEN country = "Other" THEN 1
                   ELSE 0
                END AS sort
          FROM other_countries
         GROUP BY 1
         ORDER BY sort ASC, 2 DESC);

Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
United Kingdom,3,245.52,81.84,8.768571428571429
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


We have ordered our table by total number of customers, with "Other" at the bottom of the table. We can see that there are 15 countries with only 1 customer. Even though Czech Republic only has 2 customers, average revenue generated by these two customers seem to be the highest among all countries, followed by Portugal and India.

### As our final mission, we will find out what percentage of purchases are album purchases or individual track purchases.

"The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue."

In [6]:
%%sql
WITH invoice_tracks AS (SELECT MAX(il.track_id) max_id,
                             il.invoice_id
                        FROM invoice_line il
                       INNER JOIN track t ON t.track_id = il.track_id
                       GROUP BY il.invoice_id)

SELECT purchased_an_album,
       COUNT(DISTINCT invoice_id) num_invoices,
       CAST(COUNT(DISTINCT invoice_id) AS FLOAT)/(SELECT COUNT(*) FROM invoice) percent_invoices
  FROM (SELECT it.*,
               CASE
                   WHEN
                        (
                         SELECT t.track_id
                           FROM track t
                          WHERE t.album_id = (
                                            SELECT t1.album_id
                                              FROM track t1
                                             WHERE t1.track_id = it.max_id
                                           )
                         
                         EXCEPT
                         
                         SELECT il.track_id
                           FROM invoice_line il
                          WHERE il.invoice_id = it.invoice_id
                        ) IS NULL
                    AND
                        (
                         SELECT il.track_id
                           FROM invoice_line il
                          WHERE il.invoice_id = it.invoice_id
                         
                         EXCEPT
                         
                         SELECT t.track_id
                           FROM track t
                          WHERE t.album_id = (
                                            SELECT t1.album_id
                                              FROM track t1
                                             WHERE t1.track_id = it.max_id
                                           )
                        ) IS NULL
                    THEN "Yes" ELSE "No"
                 END AS "purchased_an_album"
            FROM invoice_tracks it)
 GROUP BY purchased_an_album;

Done.


purchased_an_album,num_invoices,percent_invoices
No,500,0.8143322475570033
Yes,114,0.18566775244299674


#### What we've done here is that we listed tracks from albums and invoices using the last track of the albums to identify them. Then, we compared the lists to find whether the customer purchased the entire album or purchased individual tracks.

As we can see, 81% of the purchases made are individual track purchases. However, 20% of our revenue still comes from entire album sales. Therefore, the new strategy that the management is considering has the possibility to cost us 1/5 of the revenue.